<a href="https://colab.research.google.com/github/Charlene393/3D-house/blob/main/AnxietyAgentTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets transformers pandas numpy matplotlib seaborn tqdm scikit-learn wordcloud

In [8]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from collections import defaultdict, Counter
from wordcloud import WordCloud
import warnings
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm


In [9]:
warnings.filterwarnings('ignore')

In [10]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# **Enhanced Dataset Loader**

In [16]:
class OptimizedDatasetLoader:
  def __init__(self):
        self.datasets = {}

  def _contains_anxiety_keywords(self, text):
      """Check if text contains anxiety-related keywords."""
      anxiety_keywords = [
          "anxiety", "panic", "fear", "worry", "nervous",
          "overthinking", "restless", "stressed", "stress", "tension"
      ]
      pattern = r"\b(" + "|".join(anxiety_keywords) + r")\b"
      return bool(re.search(pattern, text.lower()))
  def load_mentalchat16k(self):
      """
      Load MentalChat16K - Primary recommended dataset.
      Source: ShenLab/MentalChat16K (HuggingFace)
      License: CC BY 4.0
      """

      print("\n" + "=" * 70)
      print("Loading MentalChat16K Dataset")
      print("=" * 70)
      print("Source: ShenLab/MentalChat16K (HuggingFace)")
      print("License: CC BY 4.0 (Open)")
      print("Content: Anxiety, depression, grief conversations\n")

      try:
          # Load from HuggingFace
          dataset = load_dataset("ShenLab/MentalChat16K")

          processed = []
          for split in dataset.keys():
              print(f"\nProcessing split: {split}...")
              for item in dataset[split]:
                  # Handle possible variations in dataset field names
                  question = (
                      item.get("question")
                      or item.get("input")
                      or item.get("user_message")
                      or ""
                  )
                  answer = (
                      item.get("answer")
                      or item.get("output")
                      or item.get("counselor_message")
                      or ""
                  )
                  context = item.get("context", "")

                  # Basic quality filter (remove short or incomplete pairs)
                  if len(question.strip()) > 10 and len(answer.strip()) > 10:
                      processed.append({
                          "question": question.strip(),
                          "answer": answer.strip(),
                          "context": context.strip(),
                          "source": "MentalChat16K",
                          "quality": "high",
                          "anxiety_related": self._contains_anxiety_keywords(
                              question + " " + answer
                          ),
                      })

          # Convert to pandas DataFrame
          df = pd.DataFrame(processed)
          self.datasets["mentalchat16k"] = df

          print(f"\n Successfully loaded {len(df):,} conversations")
          print(f"  Anxiety-related: {df['anxiety_related'].sum():,}")

          return df

      except Exception as e:
          print(f"\n Error loading MentalChat16K: {e}")
          print("  Check internet connection and HuggingFace access")
          return None


In [18]:
df.head()

NameError: name 'df' is not defined

In [20]:
loader = OptimizedDatasetLoader()
df = loader.load_mentalchat16k()
df.head()


Loading MentalChat16K Dataset
Source: ShenLab/MentalChat16K (HuggingFace)
License: CC BY 4.0 (Open)
Content: Anxiety, depression, grief conversations


Processing split: train...

 Successfully loaded 16,055 conversations
  Anxiety-related: 9,903


,question,answer,context,source,quality,anxiety_related
0,I've been struggling with my mental health for...,I understand that you've been dealing with a s...,,MentalChat16K,high,False
1,I've been feeling overwhelmed with my caregivi...,"Your situation is complex, and it's important ...",,MentalChat16K,high,True
2,I've been feeling constantly anxious and unabl...,I can see that you're dealing with a great dea...,,MentalChat16K,high,True
3,"My mom has Alzheimer's, and I've been her prim...",I'm sorry to hear that your siblings' demands ...,,MentalChat16K,high,False
4,"I've tried setting boundaries, but it feels li...","Your concerns are valid, and it's crucial to p...",,MentalChat16K,high,True
